In [2]:
# Support Vector Machines

# SVMs can be used for linear or nonlinear classification
# regression, and outlier detection

# Linear SVM
# creates a decision boundary with largest possible margin,
# margin is distance from line to nearest point of class
# having a margin allows for new instances to be classified better
# than if the separating hyperplane had a smaller margin
# svms are sensitive to feature scaling, use some scaler on data
# finds the widest possible street(margin), the linstorylines going through nearest
# points are called support vectors
# adding more instances outside of the street in training will not affect 
# decision boundary at all, only in the street (between support vectors)

In [3]:
# Soft Margin Classification

# hard margin classification is when all instances are off of 
# the street and are on the right side
# issues: will only work if data is linearly separable
# it is sensitive to outliers

# we can avoid these issues with a more flexible model
# find a good balance between keeping the margin wide and limiting
# margin violations. this is soft margin classification

# svm in scikit, we can specify hyperparams
# C value is how much we want to avoid misclassifying each example
# higher could potentially overfit

# we want fewer margin violations, but keep in mind bias/variance
# tradeoff in order to generalize better

# load iris, scale features, train linear SVM

import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris['data'][:, (2, 3)] # petal length and width
y = (iris['target'] == 2).astype(np.float64) # Iris virginica

svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C = 1, loss = 'hinge'))
])

svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [4]:
svm_clf.predict([[5.5, 1.7]])
# does not output a probability like logistic regression,
# but simply classifies

# instead of LinearSVC, we could use SVC class with linear kernel
# or SGDClassifier with hinge loss and alpha = 1/(m*C)
# it will not converge as fast but will be useful to handle online
# classification tasks or huge datasets that don't fit in memory

# LinearSVC regularizes bias term, so center training set by first subtracting
# the mean. StandardScaler does this. hinge loss is not default, so remember to set it
# for performance, set dual to false unless there are more features than training
# instances. 

array([1.])

In [9]:
# Nonlinear SVM Classification

# you could add more features like we did previously, (x, x^2)

# we can do this in scikit with pipeline of PolynomialFeatures, 
# StandardScaler, and LinearSVC

# test on moons dataset, data points are shaped as two
# interleaving half-circles

from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples = 100, noise = .15)
polynomial_svm_clf = Pipeline([
    ('poly_features', PolynomialFeatures(degree = 3)),
    ('scaler', StandardScaler()),
    ('svm_clf', LinearSVC(C = 10, loss = 'hinge'))
])

polynomial_svm_clf.fit(X, y)


# adding polynomial features is simple and can work greate with ml algorithms
# low degree will not likely help with complex data, and high degree creates
# too many features making the model too slow

# there is a kernel trick for SVM that appears as if you added many polynomial
# features, without having to add them. therefore there is no combinatorial
# explosion

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

In [6]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel = 'poly', degree = 3, coef0 = 1, C = 5))
])
poly_kernel_svm_clf.fit(X, y)

# this trains a SVM using third degree polynomial kernel. higher 
# degree will overfit, lower degree will underfit. coef0 controls how
# much the model is influenced by high degree polynomials versus
# low degree polynomials

# can find best hyperparameters using grid search (chapter 2)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

In [8]:
# Can also transform nonlinear data into linear data
# by using a Similarity function like Gaussian
# Radial Basis Function
# plug into this function to compute new features

# phi(x, l) = exp(-y||x-l||^2)
# after transforming data, it will be linearly separable
# this can cause high dimensionality though, because
# it will create a landmark for every data instance

# a landmark is an object to compare to the data instance
# and tell how similar it is

# similar to polynomial features method, this can be used
# with any ml algorithm but can be computationally expensive

# the kernel trick is at play here, making it possible to 
# have a similar result as if you had added many similarity features

rbf_kernel_svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel = 'rbf', gamma = 5, C = .001))
])
rbf_kernel_svm_clf.fit(X, y)

# increasing gamma makes bell curve narrower
# this in turn makes the decision boundary change
# high gamma will be more complex and irregular,
# low gamma will be more smooth and easy

# gamma is hyperparam, if overfitting you should reduce it,
# if underfitting, you should increase it

# C param tells how strict you are on misclassification,
# the higher it is, the closer the decision boundary will be
# to encompassing all correct points on training data

# other kernels exist but are far less common
# there is string kernel for classifying documents or DNA
# sequences, based on Levenshtein Distance (single character edit(insert, delete, swaps) required
# to change one word into another)

# to choose kernel, always try LinearSVC before SVC(kernel = 'linear') cause LinearSVC is faster
# if training set not too large, try Gaussian rbf. If extra computing power, use cross validation
# and grid search to find best combo

# LinearSVC is O(m * n) and algorithm will take longer according to tolerance parameter
# in most classifications, default tolerance is fine. does not support kernel trick

# SVC supports kernel trick, but it is  between O(m^2 * n) O(m^3 * n). oof
# m is training instances, n is number of features. algorithm is good for
# complex small or medium training data. scales well with sparse features
# (each instance has few nonzero features)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

In [11]:
# SVM Regression

# svm can also be used for linear and nonlinear regression
# objective is to fit as many instances as possible on the
# street (in margin) while limiting margin violations
# width of margin is hyperparameter epsilon. adding more
# training examples within the margin does not affect
# the model's prediction, thus the model is epsilon insensitive

# note you should center and scale data first

from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon = 1.5)
svm_reg.fit(X, y)

# for nonlinear regression, use kernelized svm model

# regularization again dependent on C, high C value is little,
# low C value is a lot
# low c value is large margin, high c value is small margin.
# therefore regularizes by telling how wrong you can be

from sklearn.svm import SVR

svm_poly_reg = SVR(kernel = 'poly', degree = 2, epsilon = .1)
svm_poly_reg.fit(X, y)



SVR(degree=2, kernel='poly')

In [12]:
# how does it work?

# the linear svm predicts using a sign function of w^Tx + b
# if it is less than 0, then negative class. if greater, positive class

# for the data with petal width and length, it forms two planes
# and the decision boundary is the intersection of those two planes
# the dashed lines are where the decision function equals positive or negative classes
# forming a margin. the goal of training is to find values of w and b that
# make the margin as wide as possible while avoiding or limiting margin violations

# the overall goal is to minimize the weight vector's magnitude. this is because
# the slope of the decision boundary is equal to the magnitude of the weight vector.
# therefore, smaller weight vector means wider margin. also subject to the constraint of
# wanting greater than 1 for positive instances, less than -1 for negative training instances

# therefore
# minimize 1/2 w^Tw
# subject to t^(i)(w^Tx^(i) + b) >= 1 for i = 1, 2, ..., m
# (we add the 1/2 because it has an easy derivative, just w! while magnitude w is not differentiable
# at w = 0. optimization algorithms work better on differentiable functions)

# for soft margin, we introduce slack variable zeta^(i) which measures how much the ith instance
# is allowed to violate the margin

# so now the objectives conflict; make the slack variable as small as possible and make 1/2w^Tw as small
# as possible to increase the width of the margin. This is where the C value comes in, C is the tradeoff
# ratio of the two objectives. so the constrained optimization problem for soft svm is as follows

# minimize 1/2w^Tw + C sum(all slack variables)
# subject to t^(i)(w^Tx^(i) + b) >= 1 - slack at ith instance and slack at i >= 0 for i = 1, 2, ... m

# hard and soft margin problems are convex quadratic optimization problems with linear constraints.
# they are known as Quadratic Programming (QP) problems. there are solutions to these problems
# outside the scope of this material

# general form of qp
# minimize x subject to y where z, usually having to do with matrices

# one way to train hard margin linear svm is to use off shelf qp solver and pass it the right parameters

# The Dual Problem

# given a constrained optimization problem known as the primal problem, it is possible to express it as a different
# but closely related problem called the dual problem. The solution of the dual problem typically gives
# a lower bound to the solution of the primal problem, but under some conditions can give the same solution
# as the primal problem. in SVM, the dual and primal problems have the same solution. also, the dual problem
# makes the kernel trick possible, while the primal problem does not. 

# Kernelized SVMs (kernel trick)

# when you transform dimensions, the dot product of the transformed vectors is just equal to the square of the 
# dot product of the original vectors. 
# phi(a)^T phi(b) = (a^Tb)^2
# so instead of transforming the training data, just repalce the dot product by its square
# a kernel in machine learning is a function capable of computing the dot product of vectors
# based only on the original vectors a and b without having to compute or know about the transformation
# phi. common kernels include linear, polynomial, gaussian rbf, and sigmoid

# follows Mercer's Theorem. if a function represents a few mathematicla conditions called Mercer's Conditions
# (k is continuous and symmetric in its arguments, etc) then there is a function phi that maps a and b into another
# space. so you can use the function as a kernel because you know phi exists, but you don't even have to know what
# it is. some frequently used kernels (like sigmoid) don't respect all of Mercer's conditions, but they 
# work well in practice anyways

# Online SVMs

# one method for linear svc is to use gradient descent, but this converges slower than using qp. 

# Hinge Loss
# max (0, 1 - t). it is 0 when t >= 1. its derivative (slope) is equal to -1 if t < 1 and 0 if t > 1.
# it isn't differentiable at t = 1, but you can still use gradient descent by using any subderivative at t = 1.

# you can also implement online kernelizd svms, they are implemented in matlab and C++. for large scale nonlinear
# problems, you should probably use neural networks instead.